In [1]:
%load_ext autoreload
%aimport os, pandas, numpy, pickle
%autoreload 2

In [2]:
import sys
import os
import pickle
import argparse
import pandas as pd
from sklearn.ensemble.forest import ExtraTreesRegressor

from fqi.et_tuning import run_tuning
from fqi.reward_function import *
from fqi.sars_creator import to_SARS
from fqi.utils import *

from trlib.policies.valuebased import EpsilonGreedy, Softmax
from trlib.policies.qfunction import ZeroQ
from trlib.algorithms.reinforcement.fqi_driver import FQIDriver, DoubleFQIDriver
from trlib.environments.trackEnv import TrackEnv
from trlib.utilities.ActionDispatcher import *

from fqi.fqi_evaluate import run_evaluation

/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
simulations = pd.read_csv('./trajectory/dataset_offroad.csv',
                              dtype={'isReference': bool, 'is_partial':bool})

#simulations = simulations[(simulations['NLap'] == 1) | (simulations['NLap'] == 9)]

ref_tr = pd.read_csv('./trajectory/ref_traj.csv')
ref_tr.columns = ['time', 'Acceleration_x', 'Acceleration_y', 'speed_x', 'speed_y',
       'xCarWorld', 'yCarWorld', 'alpha_step']

In [4]:
file_name = 'model_file/penalty/penalty.pkl'
with open('./' + file_name, 'rb') as pen:
     penalty = pickle.load(pen)

In [40]:
def compute_offroad_penalty(X, trackPos):
        logp = penalty.kde.score_samples(X)
        #penalty.alpha * logp + penalty.scale_f
        mask = np.absolute(trackPos) > 1
        trackPos[~mask] = 0
        trackPos[mask] = np.absolute(trackPos[mask])*100
        return penalty.alpha * logp + penalty.scale_f + trackPos, trackPos

In [6]:
rf = Temporal_projection(ref_tr, penalty=penalty, clip_range=(-np.inf, np.inf))

In [34]:
data = pd.read_csv('./trajectory/dataset_offroad.csv',
                              dtype={'isReference': bool, 'is_partial':bool})
NLap = 12
data = data[data['NLap'] == NLap]
episodes = {}
for e in np.unique(data['NLap']):
    mask = data['NLap'] == e
    lap_df = data[mask]
    n_samples = np.count_nonzero(mask)
    # create timestamp column
    timestamp = pd.DataFrame({'t': np.zeros([n_samples - 1])})
    # Create NLap column
    nlap = pd.DataFrame({'NLap': np.ones([n_samples - 1]) * e}, dtype=int)
    # Add state and action
    state_action = lap_df[state_cols+action_cols].iloc[:-1].reset_index(drop=True)
    # Add reward
    #reward = pd.DataFrame({'r': reward_function(lap_df)})
    r = rf(lap_df)
    #print(r)
    print('type:', type(r))
    print('shape', r.shape)

type: <class 'numpy.ndarray'>
shape (849,)


In [41]:
penalty.kernel
penalty.bandwidth
off_p, trackPos = compute_offroad_penalty(data[state_cols].values[1:], data['trackPos'].values[1:])

In [42]:
pd.DataFrame(trackPos, columns=['trackPos']).describe()

,trackPos
count,849.000000
mean,44.769514
std,143.334689
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1007.229055
